# CNC Mill Tool Wear
- https://www.kaggle.com/shasun/tool-wear-detection-in-cnc-mill
- Predict "Machining_Process"

## Setup

In [84]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [85]:
import os
path_parent = os.path.dirname(os.getcwd())
saveFolder = os.path.join(path_parent, 'Topic 02 - Classification', 'Data_CNC_Processed')
print(saveFolder)
#saveFolder = '..\Topic 02 - Classification\Data_CNC_Processed'
filename = 'pred_machining_process'
compression = 'gzip'
df = pd.read_pickle(f'{saveFolder}/{filename}.pkl',compression=compression)
df.head(3)

f:\Work\Coding\Class\ai-class-2564-1\Topic 02 - Classification\Data_CNC_Processed


,X1_ActualPosition,X1_ActualVelocity,X1_ActualAcceleration,X1_CommandPosition,X1_CommandVelocity,X1_CommandAcceleration,X1_CurrentFeedback,X1_DCBusVoltage,X1_OutputCurrent,X1_OutputVoltage,...,S1_OutputCurrent,S1_OutputVoltage,S1_OutputPower,S1_SystemInertia,M1_CURRENT_PROGRAM_NUMBER,M1_sequence_number,M1_CURRENT_FEEDRATE,Machining_Process,feedrate,clamp_pressure
0,198.0,0.0,0.00,198.0,0.0,0.000000,0.18,0.0207,329.0,2.77,...,329.0,0.0,6.960000e-07,12.0,1.0,0.0,50.0,7,6,4.0
1,198.0,-10.8,-350.00,198.0,-13.6,-358.000000,-10.90,0.1860,328.0,23.30,...,328.0,0.0,-5.270000e-07,12.0,1.0,4.0,50.0,7,6,4.0
2,196.0,-17.8,-6.25,196.0,-17.9,-0.000095,-8.59,0.1400,328.0,30.60,...,328.0,0.0,9.100000e-07,12.0,1.0,7.0,50.0,7,6,4.0


In [86]:
max_features = 20

# Transform X with variance selector

In [87]:
from sklearn.feature_selection import VarianceThreshold

colsX = np.array([item for item in df.columns if item not in 'Machining_Process'])
X = df[colsX].values
y = df['Machining_Process'].values

# Creating object
sel = VarianceThreshold(threshold=0)

# Fitting
sel.fit(X)

# Boolean
sup = sel.get_support()
cols_sel = colsX[sup]
print(f'Chosen columns:')
print('-'*30)
print(*cols_sel, sep=', ')

cols_rem = colsX[np.logical_not(sup)]
print(f'\nRemoved columns:')
print('-'*30)
print(*cols_rem, sep=', ')

X = sel.transform(X)
dfX = pd.DataFrame(data=X, columns=cols_sel)
dfX.head()

# Initialize dictionary to store selector
sel = {}

Chosen columns:
------------------------------
X1_ActualPosition, X1_ActualVelocity, X1_ActualAcceleration, X1_CommandPosition, X1_CommandVelocity, X1_CommandAcceleration, X1_CurrentFeedback, X1_DCBusVoltage, X1_OutputCurrent, X1_OutputVoltage, X1_OutputPower, Y1_ActualPosition, Y1_ActualVelocity, Y1_ActualAcceleration, Y1_CommandPosition, Y1_CommandVelocity, Y1_CommandAcceleration, Y1_CurrentFeedback, Y1_DCBusVoltage, Y1_OutputCurrent, Y1_OutputVoltage, Y1_OutputPower, Z1_ActualPosition, Z1_ActualVelocity, Z1_ActualAcceleration, Z1_CommandPosition, Z1_CommandVelocity, Z1_CommandAcceleration, S1_ActualPosition, S1_ActualVelocity, S1_ActualAcceleration, S1_CommandPosition, S1_CommandVelocity, S1_CommandAcceleration, S1_CurrentFeedback, S1_DCBusVoltage, S1_OutputCurrent, S1_OutputVoltage, S1_OutputPower, M1_CURRENT_PROGRAM_NUMBER, M1_sequence_number, M1_CURRENT_FEEDRATE, feedrate, clamp_pressure

Removed columns:
------------------------------
Z1_CurrentFeedback, Z1_DCBusVoltage, Z1_Outp

## L1

In [88]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=0, stratify=y)
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
print(X_train_std.shape)
# Classifier
lr = LogisticRegression(solver='liblinear', penalty='l1', C=0.1, multi_class='ovr')

# Create selector object
threshold=1e-5
sel['L1'] = SelectFromModel(estimator=lr, norm_order=1, threshold=threshold, max_features=max_features)

# Training
sel['L1'].fit(X_train_std, y_train)

# Extract norm of weights
coef = sel['L1'].estimator_.coef_
coef_norm = np.linalg.norm(coef , ord=1, axis=0)

# Get column boolean
sup = sel['L1'].get_support()
print('\nSupport array')
print('-'*30)
print(*sup, sep=', ')

(2528, 44)

Support array
------------------------------
True, False, True, False, False, False, True, True, True, False, True, False, False, False, False, True, False, True, True, True, False, True, False, False, True, True, False, False, True, False, False, True, False, False, False, False, True, False, False, False, True, True, True, True


In [89]:
# Sort array from based on weights from large to small
cols = dfX.columns
idxs = np.argsort(coef_norm)[::-1]
sup = sup[idxs]
coef_norm = coef_norm[idxs]
cols = cols[idxs]

# Print results
for count, (col, coef) in enumerate(zip(cols, coef_norm)):
    print(f"{count+1:2d}) {col:30s} \t{coef:5.3f}")

 1) feedrate                       	1.117
 2) clamp_pressure                 	0.728
 3) Y1_OutputCurrent               	0.704
 4) X1_OutputCurrent               	0.518
 5) X1_ActualPosition              	0.488
 6) M1_CURRENT_FEEDRATE            	0.478
 7) M1_sequence_number             	0.424
 8) S1_CommandPosition             	0.386
 9) S1_OutputCurrent               	0.341
10) Y1_CurrentFeedback             	0.319
11) Y1_OutputPower                 	0.282
12) X1_OutputPower                 	0.281
13) Y1_DCBusVoltage                	0.244
14) X1_ActualAcceleration          	0.225
15) Z1_ActualAcceleration          	0.204
16) Z1_CommandPosition             	0.202
17) S1_ActualPosition              	0.197
18) X1_CurrentFeedback             	0.193
19) Y1_CommandVelocity             	0.189
20) X1_DCBusVoltage                	0.163
21) Z1_ActualVelocity              	0.162
22) Y1_CommandPosition             	0.153
23) X1_ActualVelocity              	0.149
24) X1_CommandAcceleration        

In [90]:
# Print columns
cols_sel = cols[sup]
print('\nChosen columns')
print('-'*30)
print(*cols_sel, sep=', ')

cols_rem = cols[np.logical_not(sup)]
print(f'\nRemoved columns')
print('-'*30)
print(*cols_rem, sep=', ')


Chosen columns
------------------------------
feedrate, clamp_pressure, Y1_OutputCurrent, X1_OutputCurrent, X1_ActualPosition, M1_CURRENT_FEEDRATE, M1_sequence_number, S1_CommandPosition, S1_OutputCurrent, Y1_CurrentFeedback, Y1_OutputPower, X1_OutputPower, Y1_DCBusVoltage, X1_ActualAcceleration, Z1_ActualAcceleration, Z1_CommandPosition, S1_ActualPosition, X1_CurrentFeedback, Y1_CommandVelocity, X1_DCBusVoltage

Removed columns
------------------------------
Z1_ActualVelocity, Y1_CommandPosition, X1_ActualVelocity, X1_CommandAcceleration, Y1_ActualVelocity, M1_CURRENT_PROGRAM_NUMBER, S1_CommandAcceleration, Y1_ActualAcceleration, X1_OutputVoltage, S1_ActualAcceleration, S1_CurrentFeedback, Y1_CommandAcceleration, Y1_OutputVoltage, X1_CommandPosition, S1_ActualVelocity, Z1_CommandVelocity, Z1_CommandAcceleration, X1_CommandVelocity, S1_CommandVelocity, S1_DCBusVoltage, Y1_ActualPosition, S1_OutputVoltage, Z1_ActualPosition, S1_OutputPower


## Importance

In [91]:
from sklearn.ensemble import RandomForestClassifier

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=0, stratify=y)
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
print(X_train_std.shape)

#Create classifier
forest = RandomForestClassifier(n_estimators=500, random_state=0, n_jobs=-1)

#Create object
threshold = 1e-15
sel['imp'] = SelectFromModel(forest, threshold=threshold, max_features=max_features)

# Training (Actually, it does not matter whether we have X_train_std or X_train because random forrest does not care)
sel['imp'].fit(X_train_std, y_train)

# Columns chosen
sup = sel['imp'].get_support()

# Extract importances values
importances = sel['imp'].estimator_.feature_importances_

# Select only chosen columns
print('\nSupport array')
print('-'*30)
print(*sup, sep=', ')

(2528, 44)

Support array
------------------------------
False, True, True, False, False, False, True, True, False, True, True, False, True, True, False, False, False, True, True, False, True, True, False, False, False, False, False, False, True, False, True, True, False, False, True, True, True, False, True, False, True, False, False, False


In [92]:
# Sort array from based on importances from large to small
cols = dfX.columns
idxs = np.argsort(importances)[::-1]
sup = sup[idxs]
importances = importances[idxs]
cols = cols[idxs]

# Print results
for count, (col, importance) in enumerate(zip(cols, importances)):
    print(f"{count+1:2d}) {col:30s} \t{importance:5.3f}")

 1) S1_CommandPosition             	0.043
 2) S1_ActualPosition              	0.043
 3) Y1_CurrentFeedback             	0.042
 4) S1_CurrentFeedback             	0.039
 5) X1_OutputPower                 	0.038
 6) X1_CurrentFeedback             	0.038
 7) Y1_OutputPower                 	0.038
 8) S1_ActualAcceleration          	0.037
 9) Y1_DCBusVoltage                	0.035
10) Y1_OutputVoltage               	0.035
11) X1_OutputVoltage               	0.035
12) X1_DCBusVoltage                	0.035
13) S1_OutputPower                 	0.035
14) X1_ActualAcceleration          	0.031
15) X1_ActualVelocity              	0.030
16) Y1_ActualAcceleration          	0.029
17) M1_sequence_number             	0.029
18) Y1_ActualVelocity              	0.028
19) S1_OutputCurrent               	0.025
20) S1_DCBusVoltage                	0.023
21) Y1_CommandPosition             	0.020
22) Y1_ActualPosition              	0.020
23) Z1_ActualAcceleration          	0.020
24) Z1_CommandPosition            

In [93]:
# Print columns
cols_sel = cols[sup]
print('\nChosen columns')
print('-'*30)
print(*cols_sel, sep=', ')

cols_rem = cols[np.logical_not(sup)]
print(f'\nRemoved columns')
print('-'*30)
print(*cols_rem, sep=', ')


Chosen columns
------------------------------
S1_CommandPosition, S1_ActualPosition, Y1_CurrentFeedback, S1_CurrentFeedback, X1_OutputPower, X1_CurrentFeedback, Y1_OutputPower, S1_ActualAcceleration, Y1_DCBusVoltage, Y1_OutputVoltage, X1_OutputVoltage, X1_DCBusVoltage, S1_OutputPower, X1_ActualAcceleration, X1_ActualVelocity, Y1_ActualAcceleration, M1_sequence_number, Y1_ActualVelocity, S1_OutputCurrent, S1_DCBusVoltage

Removed columns
------------------------------
Y1_CommandPosition, Y1_ActualPosition, Z1_ActualAcceleration, Z1_CommandPosition, Z1_ActualPosition, X1_ActualPosition, clamp_pressure, X1_CommandPosition, S1_ActualVelocity, S1_OutputVoltage, Z1_ActualVelocity, Y1_OutputCurrent, X1_CommandVelocity, X1_OutputCurrent, Y1_CommandVelocity, Y1_CommandAcceleration, X1_CommandAcceleration, feedrate, S1_CommandAcceleration, M1_CURRENT_FEEDRATE, M1_CURRENT_PROGRAM_NUMBER, Z1_CommandVelocity, Z1_CommandAcceleration, S1_CommandVelocity


# Sequential feature selector
- KNN yields much worse accuracy
- Better results with small forrest as classifier
- Backward yields better accuracy


In [94]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SequentialFeatureSelector

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.98, random_state=0, stratify=y)
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
print(X_train_std.shape)

# Create classifier
knn = KNeighborsClassifier(n_neighbors=3)
forest = RandomForestClassifier(n_estimators=10, random_state=0, n_jobs=-1)

# Create selector object
sel['seq_forward'] = SequentialFeatureSelector(forest, direction='forward', n_features_to_select=max_features, scoring='accuracy', cv=3, n_jobs=-1)

# Training
sel['seq_forward'].fit(X_train_std, y_train)

# Get column boolean
sup = sel['seq_forward'].get_support()
print('\nSupport array')
print('-'*30)
print(*sup, sep=', ')

(505, 44)

Support array
------------------------------
False, False, False, False, False, True, False, False, True, False, False, True, True, False, True, True, False, False, False, True, True, False, True, True, False, True, True, True, True, False, False, True, True, False, False, False, False, True, False, True, False, False, True, True


In [95]:
# Print columns
cols = dfX.columns
cols_sel = cols[sup]
print('\nChosen columns')
print('-'*30)
print(*cols_sel, sep=', ')

cols_rem = cols[np.logical_not(sup)]
print(f'\nRemoved columns')
print('-'*30)
print(*cols_rem, sep=', ')


Chosen columns
------------------------------
X1_CommandAcceleration, X1_OutputCurrent, Y1_ActualPosition, Y1_ActualVelocity, Y1_CommandPosition, Y1_CommandVelocity, Y1_OutputCurrent, Y1_OutputVoltage, Z1_ActualPosition, Z1_ActualVelocity, Z1_CommandPosition, Z1_CommandVelocity, Z1_CommandAcceleration, S1_ActualPosition, S1_CommandPosition, S1_CommandVelocity, S1_OutputVoltage, M1_CURRENT_PROGRAM_NUMBER, feedrate, clamp_pressure

Removed columns
------------------------------
X1_ActualPosition, X1_ActualVelocity, X1_ActualAcceleration, X1_CommandPosition, X1_CommandVelocity, X1_CurrentFeedback, X1_DCBusVoltage, X1_OutputVoltage, X1_OutputPower, Y1_ActualAcceleration, Y1_CommandAcceleration, Y1_CurrentFeedback, Y1_DCBusVoltage, Y1_OutputPower, Z1_ActualAcceleration, S1_ActualVelocity, S1_ActualAcceleration, S1_CommandAcceleration, S1_CurrentFeedback, S1_DCBusVoltage, S1_OutputCurrent, S1_OutputPower, M1_sequence_number, M1_CURRENT_FEEDRATE


In [96]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SequentialFeatureSelector

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.98, random_state=0, stratify=y)
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
print(X_train_std.shape)

# Create classifier
knn = KNeighborsClassifier(n_neighbors=3)
forest = RandomForestClassifier(n_estimators=10, random_state=0, n_jobs=-1)

# Create selector object
sel['seq_backward'] = SequentialFeatureSelector(forest, direction='backward', n_features_to_select=max_features, scoring='accuracy', cv=3, n_jobs=-1)

# Training
sel['seq_backward'].fit(X_train_std, y_train)

# Get column boolean
sup = sel['seq_backward'].get_support()
print('\nSupport array')
print('-'*30)
print(*sup, sep=', ')

(505, 44)

Support array
------------------------------
False, False, False, True, False, False, False, False, True, False, True, True, False, True, False, True, True, True, False, True, False, True, True, False, True, False, False, False, True, False, False, True, False, False, False, False, True, False, True, True, True, False, True, True


## Training with random forrest

In [97]:
from sklearn.ensemble import RandomForestClassifier

paramSetAll = {
    "ex1": {
        "criterion": "gini",
        "n_estimators": 25,
        "max_samples": None,
        "max_features": "auto",
        "max_depth": None,
    },
    "ex2": {
        "criterion": "gini",
        "n_estimators": 50,
        "max_samples": None,
        "max_features": "auto",
        "max_depth": None,
    },
    "ex3": {
        "criterion": "gini",
        "n_estimators": 100,
        "max_samples": None,
        "max_features": "auto",
        "max_depth": None,
    },
}

In [98]:
cl = list(range(1,4))
ca = [ f'ex{i}' for i in cl]
paramSet = { k: paramSetAll[k] for k in ca} 
print(paramSet)

{'ex1': {'criterion': 'gini', 'n_estimators': 25, 'max_samples': None, 'max_features': 'auto', 'max_depth': None}, 'ex2': {'criterion': 'gini', 'n_estimators': 50, 'max_samples': None, 'max_features': 'auto', 'max_depth': None}, 'ex3': {'criterion': 'gini', 'n_estimators': 100, 'max_samples': None, 'max_features': 'auto', 'max_depth': None}}


In [99]:
def training(X_train, y_train, X_test, y_test):
    for paramName, paramValue in paramSet.items():
        # Extract parameteres
        forrest = RandomForestClassifier(**paramValue)

        # Training
        forrest.fit(X_train, y_train)

        # Prediction
        y_pred = forrest.predict(X_test)

        # Misclassification from the test samples
        sumMiss = (y_test != y_pred).sum()

        # Accuracy score from the test samples
        accuracyScore = accuracy_score(y_test, y_pred)

        print(f"Parameters: {paramValue}")
        print(f"Misclassified examples: {sumMiss}")
        print(f"Accuracy score: {accuracyScore}")
        print(f"--------------------------------------------------")

In [100]:
# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

# Standardize
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)
print(X_train_std.shape)

(17700, 44)


In [101]:
# No transformation
training(X_train_std, y_train, X_test_std, y_test)

Parameters: {'criterion': 'gini', 'n_estimators': 25, 'max_samples': None, 'max_features': 'auto', 'max_depth': None}
Misclassified examples: 2416
Accuracy score: 0.681518586870551
--------------------------------------------------
Parameters: {'criterion': 'gini', 'n_estimators': 50, 'max_samples': None, 'max_features': 'auto', 'max_depth': None}
Misclassified examples: 2311
Accuracy score: 0.6953598734510941
--------------------------------------------------
Parameters: {'criterion': 'gini', 'n_estimators': 100, 'max_samples': None, 'max_features': 'auto', 'max_depth': None}
Misclassified examples: 2162
Accuracy score: 0.7150013182177696
--------------------------------------------------


In [102]:
X_train_std_trans = sel['L1'].transform(X_train_std)
X_test_std_trans = sel['L1'].transform(X_test_std)

training(X_train_std_trans, y_train, X_test_std_trans, y_test)

Parameters: {'criterion': 'gini', 'n_estimators': 25, 'max_samples': None, 'max_features': 'auto', 'max_depth': None}
Misclassified examples: 2343
Accuracy score: 0.6911415765884524
--------------------------------------------------
Parameters: {'criterion': 'gini', 'n_estimators': 50, 'max_samples': None, 'max_features': 'auto', 'max_depth': None}
Misclassified examples: 2222
Accuracy score: 0.7070920116003163
--------------------------------------------------
Parameters: {'criterion': 'gini', 'n_estimators': 100, 'max_samples': None, 'max_features': 'auto', 'max_depth': None}
Misclassified examples: 2121
Accuracy score: 0.7204060110730293
--------------------------------------------------


In [103]:
X_train_std_trans = sel['imp'].transform(X_train_std)
X_test_std_trans = sel['imp'].transform(X_test_std)

training(X_train_std_trans, y_train, X_test_std_trans, y_test)

Parameters: {'criterion': 'gini', 'n_estimators': 25, 'max_samples': None, 'max_features': 'auto', 'max_depth': None}
Misclassified examples: 3545
Accuracy score: 0.5326918006854733
--------------------------------------------------
Parameters: {'criterion': 'gini', 'n_estimators': 50, 'max_samples': None, 'max_features': 'auto', 'max_depth': None}
Misclassified examples: 3450
Accuracy score: 0.5452148694964408
--------------------------------------------------
Parameters: {'criterion': 'gini', 'n_estimators': 100, 'max_samples': None, 'max_features': 'auto', 'max_depth': None}
Misclassified examples: 3295
Accuracy score: 0.5656472449248616
--------------------------------------------------


In [104]:
X_train_std_trans = sel['seq_forward'].transform(X_train_std)
X_test_std_trans = sel['seq_forward'].transform(X_test_std)

training(X_train_std_trans, y_train, X_test_std_trans, y_test)

Parameters: {'criterion': 'gini', 'n_estimators': 25, 'max_samples': None, 'max_features': 'auto', 'max_depth': None}
Misclassified examples: 2141
Accuracy score: 0.7177695755338782
--------------------------------------------------
Parameters: {'criterion': 'gini', 'n_estimators': 50, 'max_samples': None, 'max_features': 'auto', 'max_depth': None}
Misclassified examples: 2034
Accuracy score: 0.7318745056683365
--------------------------------------------------
Parameters: {'criterion': 'gini', 'n_estimators': 100, 'max_samples': None, 'max_features': 'auto', 'max_depth': None}
Misclassified examples: 1988
Accuracy score: 0.7379383074083838
--------------------------------------------------


In [105]:
X_train_std_trans = sel['seq_backward'].transform(X_train_std)
X_test_std_trans = sel['seq_backward'].transform(X_test_std)

training(X_train_std_trans, y_train, X_test_std_trans, y_test)

Parameters: {'criterion': 'gini', 'n_estimators': 25, 'max_samples': None, 'max_features': 'auto', 'max_depth': None}
Misclassified examples: 2110
Accuracy score: 0.7218560506195624
--------------------------------------------------
Parameters: {'criterion': 'gini', 'n_estimators': 50, 'max_samples': None, 'max_features': 'auto', 'max_depth': None}
Misclassified examples: 2054
Accuracy score: 0.7292380701291853
--------------------------------------------------
Parameters: {'criterion': 'gini', 'n_estimators': 100, 'max_samples': None, 'max_features': 'auto', 'max_depth': None}
Misclassified examples: 1998
Accuracy score: 0.7366200896388083
--------------------------------------------------
